In [0]:
##Pytorch Req Code

import torch
import scipy.sparse as sparse
import pandas as pd
import numpy as np
import requests, zipfile, io
import random, time, sys
from pandas.compat import StringIO
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
!pip install implicit

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#r = requests.get('http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip')
#z = zipfile.ZipFile(io.BytesIO(r.content))
#z.extractall()
#item = pd.read_csv("./user_artists.dat", sep='\s+')

#read data into DataFram
Main_File = './user_artists.dat'
#Lookup_File = './artists.dat'

#dfnp = np.genfromtxt(Main_File, skip_header=1) # Skips the header and put numbers into array
dfnp = pd.read_csv('https://s3.us-east-2.amazonaws.com/rudydata/user_artist.csv',sep= ",", engine='python' ).values
UATensor = torch.Tensor(dfnp) # Converts dataset to Tensor

# Setting up the sparse matrix
user_artist = UATensor[:,:2].t().type(torch.LongTensor) # Sets up the User & Artist index as Long Tensor for input to sparse function
weight = UATensor.t()[2] # Sets up the number of times a user listens to an artist as a Float Tensor for input to sparse function
max_user = int(max(user_artist[0]).item()) + 1 # Max user + 1 for sparse matrix size
max_artist = int(max(user_artist[1]).item()) + 1 # Max artist + 1 for sparse matrix size
listens_sparse = torch.sparse.FloatTensor(user_artist, weight, torch.Size([max_user, max_artist])) # Sparse Matrix; .to_dense(); .to_sparse()

nonzeros = listens_sparse._nnz() # Checks the number of non-zero items; resulted in 92,834 and matches
total_elements = listens_sparse.nelement() # Number of elements in the sparse matrix; resulted in 39,385,346
sparsity = 1 - nonzeros / total_elements # Percentage of how sparse the matrix is; resulted in 99.764293%. 
                                           # Not quite matching because some users have not listened to anything.
                                           # Should these users be excluded?

# Creating Train and Test Set
pct_test = 0.2 # The percentage of user-artist interactions that you want to mask in the training set.

# Make a copy of the original set to be the test set, and store the test set as a binary preference matrix
test_set = torch.sparse.FloatTensor(user_artist, torch.ones(weight.size()), torch.Size([max_user, max_artist]))

num_samples = torch.randperm(nonzeros) # Randomly assigns a row a number between 0 to number of idices
UATensor_smpl = UATensor[num_samples > pct_test * nonzeros] # Samples 1 - pct_test of UATensor, so that this masks pct_test of the training set
user_artist_smpl = UATensor_smpl[:,:2].t().type(torch.LongTensor) # Sets up the user_artist of the sample
weight_smpl = UATensor_smpl.t()[2] # Sets up the weight of the sample
training_set = torch.sparse.FloatTensor(user_artist_smpl, weight_smpl, torch.Size([max_user, max_artist])) # Create training set with sample size

# --- ALS Model --- #
print("begin model")

# Parameters for ALS Model
lambda_val = 0.01 # Used for regularization. Increasing this value may increase bias but decrease variance
alpha = 2 # Associated with confidence matrix where Cui = 1 + alpha*Rui. Decreasing this will decrease the variability in confidence between various ratings.
iterations = 10 # The number of times to alternate between both user feature vector and artist feature vector in ALS. 
                # More iterations will allow better convergence at the cost of increased computation.
rank_size = 15 # The number of latent features in the user/artist feature vectors. Increasing the number of features may overfit but could reduce bias.
seed = 0 # Set the seed for reproducible reseults

# Set up Confidence Matrix
conf = (alpha * training_set).to_dense().to(device) # To allow the matrix to stay sparse, I will add one later when each row is taken and converted to dense.
pref_all = conf.clone()
pref_all[pref_all != 0] = 1 # Create binarized preference vector

# initialize our X/Y feature vectors randomly with a set seed
X = torch.rand(max_user, rank_size).to(device) # Random numbers in a m x rank shape
Y = torch.rand(max_artist, rank_size).to(device) # Normally this would be rank x n but we can 
                                                  # transpose at the end. Makes calculation more simple.
X_eye = torch.eye(max_user).to(device)
Y_eye = torch.eye(max_artist).to(device)
lambda_eye = lambda_val * torch.eye(rank_size).to(device) # Our regularization term lambda*I.

# Begin iterations
for iter_step in range(iterations): # Iterate back and forth between solving X given fixed Y and vice versa
  start = time.time()
  
  # Compute yTy and xTx at beginning of each iteration to save computing time
  yTy = Y.t().mm(Y).to(device)
  xTx = X.t().mm(X).to(device)
  
  # Begin iteration to solve for X based on fixed Y
  for u in range(max_user):
    conf_samp = conf[u,:] # Grab user row from confidence matrix and convert to dense
    pref = pref_all[u,:]
    CuI = conf_samp.diag() # Get Cu - I term, don't need to subtract 1 since we never added it
    yTCuIY = Y.t().mm(CuI).mm(Y) # This is the yT(Cu-I)Y term
    yTCupu = Y.t().mm(CuI + Y_eye).mm(pref.unsqueeze(0).t()) # This is the yTCuPu term, where we add the eye back in Cu - I + I = Cu
    X[u] = (yTy + yTCuIY + lambda_eye).inverse().mm(yTCupu).t() # Solve for Xu = ((yTy + yT(Cu-I)Y + lambda*I)^-1)yTCuPu
    
  # Begin iteration to solve for Y based on fixed X 
  for a in range(max_artist):
    conf_samp = conf[:,a] # transpose to get it in row format and convert to dense
    pref =  pref_all[:,a]
    CiI = conf_samp.diag() # Get Ci - I term, don't need to subtract 1 since we never added it
    xTCiIX = X.t().mm(CiI).mm(X) # This is the xT(Cu-I)X term
    xTCiPi = X.t().mm(CiI + X_eye).mm(pref.unsqueeze(0).t()) # This is the xTCiPi term
    Y[a] = (xTx + xTCiIX + lambda_eye).inverse().mm(xTCiPi).t() # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
    
  print(iter_step, time.time() - start)
  
def auc_score(predictions, test):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)   
  
def calc_mean_auc(training_set, altered_users, predictions, test_set):
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

# To remove artists where they were never listened to from dataset
test_set_np = test_set.to_dense().numpy()
a_count = test_set_np.sum(axis=0)
a_zeros = np.where(a_count == 0)[0]

# Sets up inputs for Calc Mean AUC
train_np = sparse.csr_matrix(np.delete(training_set.to_dense().numpy(), a_zeros, axis=1))
altered_users = np.unique(UATensor[num_samples <= pct_test * nonzeros][:,0].int().numpy())
user_vecs = X.to('cpu').numpy()
item_vecs = np.delete(Y.t().to('cpu').numpy(), a_zeros, axis=1)
test_np = sparse.csr_matrix(np.delete(test_set_np, a_zeros, axis=1))

# AUC for our recommender system
calc_mean_auc(train_np, altered_users, [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs)], test_np)

begin model
